In [1]:
# -*- coding: utf-8 -*- 

import re 
from tqdm import tqdm 
import time
from datetime import datetime
import sqlite3
import sys
import os
import pandas as pd
import unify

In [2]:
def connect(file_path, primary, columns):
    con = sqlite3.connect(file_path)
    cur = con.cursor()
    cols = ", ".join([c + ' Varchar' for c in columns]) 
    cur.execute("create table meta ("+primary+" Varchar PRIMARY KEY, "+cols+" )")
    cur.execute("CREATE INDEX log on meta (textid);")
    cur.execute("create table plain_texts (id Varchar(128) NOT NULL PRIMARY KEY, text Varchar NOT NULL);")
    cur.execute("create table tagged_texts (id Varchar(128) NOT NULL PRIMARY KEY, text Varchar NOT NULL );")
    con.commit()
    return con, cur

In [3]:
workdir = r'/home/tari/Загрузки/taiga/stihi_ru'
filename = 'stihi_ru.db'
file_path = os.path.join(workdir, filename)
metatablepath = os.path.join(workdir,'newmetadata.csv')
tagged = os.path.join(workdir,'texts_tagged')
plain = os.path.join(workdir,'texts')

meta = pd.read_csv(metatablepath, sep='\t', encoding='utf8')
meta = meta.fillna('')
meta.head()

,segment,textid,textname,textregion,textrubric,textdiff,author,authortexts,authorreaders,magazine,date,time,tags,source
0,stihi_ru,201612143306,в излучине реки,,без рубрики,,Евгения Костюкова,274,233094,,14.12.2016,10:08,,http://www.stihi.ru/2016/12/14/3306
1,stihi_ru,2016121310219,облако-яблоко,,без рубрики,,Евгения Костюкова,274,233094,,13.12.2016,20:45,,http://www.stihi.ru/2016/12/13/10219
2,stihi_ru,201205184471,музыка,,без рубрики,,Евгения Костюкова,274,233094,,18.05.2012,12:35,,http://www.stihi.ru/2012/05/18/4471
3,stihi_ru,201310217439,саломея,,без рубрики,,Евгения Костюкова,274,233094,,21.10.2013,18:07,,http://www.stihi.ru/2013/10/21/7439
4,stihi_ru,201301011820,родина,,без рубрики,,Евгения Костюкова,274,233094,,01.01.2013,11:13,,http://www.stihi.ru/2013/01/01/1820


In [4]:
if not os.path.exists(filename):
    con, cur = connect(filename, meta.columns[1], [meta.columns[0]]+list(meta.columns[2:]))
else:
    con = sqlite3.connect(filename, meta.columns[1], [meta.columns[0]]+list(meta.columns[2:]))
    cur = con.cursor()

In [5]:
meta.iloc[6].to_dict()

{'author': 'Евгения Костюкова',
 'authorreaders': 233094,
 'authortexts': 274,
 'date': '01.01.2014',
 'magazine': '',
 'segment': 'stihi_ru',
 'source': 'http://www.stihi.ru/2014/01/01/2528',
 'tags': '',
 'textdiff': '',
 'textid': '201401012528',
 'textname': 'лёля',
 'textregion': '',
 'textrubric': 'белый и вольный стих',
 'time': '12:30'}

In [ ]:
for i in range(len(meta)):
    values = meta.iloc[i].to_dict()
    values['textid'] = str(values['textid'])
    values['textdiff'] = str(values['textdiff'])
    values['authorreaders'] = str(values['authorreaders'])
    values['authortexts'] = str(values['authortexts'])
    
    
    columns = ', '.join(values.keys())
    
    #print(list(values.values()))
    placeholders = ', '.join('?' * len(values))
    sql = 'INSERT INTO meta ({}) VALUES ({})'.format(columns, placeholders)
    #print(sql)
    cur.execute(sql, list(values.values()))
    
    valuest = {'id': values['textid'], 'text': unify.open_text(os.path.join(plain, str(values['textid'])+".txt"))}
    
    columns = ', '.join(valuest.keys())
    placeholders = ', '.join('?' * len(valuest))
    sql2 = 'INSERT INTO plain_texts ({}) VALUES ({})'.format(columns, placeholders) 
    cur.execute(sql2, list(valuest.values()))
    try:
        valuest2 = {'id': values['textid'], 'text': unify.open_text(open(os.path.join(tagged, str(values['textid'])+".txt"),'r', encoding='utf8').read())}
        columns = ', '.join(valuest2.keys())
        placeholders = ', '.join('?' * len(valuest2))
        sql3 = 'INSERT INTO tagged_texts ({}) VALUES ({})'.format(columns, placeholders) 
        cur.execute(sql3, list(valuest2.values()))
    except:
        valuest2 = {'id': values['textid'], 'text': ""}
        columns = ', '.join(valuest2.keys())
        placeholders = ', '.join('?' * len(valuest2))
        sql3 = 'INSERT INTO tagged_texts ({}) VALUES ({})'.format(columns, placeholders) 
        cur.execute(sql3, list(valuest2.values()))
    con.commit()